In [1]:
import os
import json

LABEL_FILE = "/mnt/c/Users/Legion/Documents/jimmy tran/Automated-invoice-processing-system/labeled_invoices_fixed.jsonl"
OCR_FOLDER = "/mnt/c/Users/Legion/Documents/jimmy tran/Automated-invoice-processing-system/easyOCR_OUTPUT"
OUTPUT_FILE = "layoutlmv3_union.jsonl"

# Bước 1: Load toàn bộ nhãn
with open(LABEL_FILE, "r", encoding="utf-8") as f:
    label_lines = [json.loads(line) for line in f]

# Bước 2: Lặp qua toàn bộ file OCR
with open(OUTPUT_FILE, "w", encoding="utf-8") as out_f:
    for idx, filename in enumerate(sorted(os.listdir(OCR_FOLDER))):
        if not filename.endswith(".json"):
            continue
        invoice_id = filename.replace(".json", "")
        ocr_path = os.path.join(OCR_FOLDER, filename)

        if idx >= len(label_lines):
            print(f"❌ Không tìm thấy label tương ứng cho {invoice_id}")
            continue

        label_obj = label_lines[idx]
        full_text = label_obj["text"]
        spans = label_obj["label"]

        char_labels = ["O"] * len(full_text)
        for start, end, tag in spans:
            char_labels[start] = f"B-{tag}"
            for i in range(start + 1, end):
                if i < len(char_labels):
                    char_labels[i] = f"I-{tag}"

        with open(ocr_path, "r", encoding="utf-8") as f:
            ocr_tokens = json.load(f)

        words, bboxes, labels = [], [], []
        offset = 0
        for token in ocr_tokens:
            word = token["text"].strip()
            if not word:
                continue
            idx_in_text = full_text.find(word, offset)
            if idx_in_text != -1:
                label = char_labels[idx_in_text]
                offset = idx_in_text + len(word)
            else:
                label = "O"
            words.append(word)
            bboxes.append(token["bbox"])
            labels.append(label)

        out_f.write(json.dumps({
            "id": invoice_id,
            "words": words,
            "bboxes": bboxes,
            "labels": labels,
            "image_path": f"converted_pngs/{invoice_id}.png"
        }, ensure_ascii=False) + "\n")

print("✅ Đã tạo xong UNION.")


✅ Đã tạo xong UNION.
